# Grading students' submissions

## Overall solution
The CLI interface for the autograder works only with docker containers.  
If we want to grade without containers, we will need to use other means.

In this notebook, we use the `grade_submission` API.  
But is is very limited:
* It grades single submissions, not batches => we need to automate for batch + use parallel threads for performance?
* It returns an object, that we will need to "parse" to reconstruct a moodle grading sheet (at least the CLI interface generates a CSV file... maybe it is possible to reuse part of the code?)

TODO: for the moment I tested grading on individual .ipynb files, need to test on zip files

## How to use it

* Go into the dist folder, edit the student version of the assignment and use the public tests
* Then try to grade it with the code below

In [ ]:
from otter.api import grade_submission
import glob # to be able to refer to the autograder zip without having to check its name (changes each time it is generated)
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [ ]:
graderzip = glob.glob("dist/autograder/assignment-autograder_*.zip")[0]
graderzip

In [ ]:
submissionlist = glob.glob("moodlesubmissions/*/*.ipynb")
submissionlist

In [ ]:
## TODO: 
# - iterate over folders in submissions and retrieve the gradebook files
# - build a feedback message with the messages of the test cases

In [ ]:
results = []
for submission in submissionlist:
    gradingoutput = grade_submission(submission, graderzip, quiet=True)
    feedbackstr = "Detailed feedback:\n" 
    for exercisename, exercise in gradingoutput.results.items():
        feedbackstr += "Exercise "+ exercise.name + ": grade = " + str(exercise.grade) + " / " + str(exercise.possible) + "\n"
    results.append([submission, gradingoutput.total, gradingoutput.possible, feedbackstr])

resultsdf = pd.DataFrame(data=results, columns=["studentname", "grade", "maxgrade", "feedbackcomments"])
pretty_print(resultsdf)